# Setup

In [1]:
import pandas as pd

TOTALS_STATS_COLUMNS_PRE = [
    'rank', 'team', 'games', 'minutes_played', 'field_goals', 'field_goals_attempted', 'field_goal_pct',
    'three_point_field_goals', 'three_point_attempts', 'three_point_pct', 'two_point_field_goals',
    'two_point_attempts', 'two_point_pct', 'free_throws', 'free_throw_attempts', 'free_throw_pct',
    'offensive_rebounds', 'defensive_rebounds', 'total_rebounds', 'assists', 'steals', 'blocks',
    'turnovers', 'personal_fouls', 'points'
]
TOTALS_STATS_COLUMNS = [f"totals_{field}" for field in TOTALS_STATS_COLUMNS_PRE]

ADVANCED_STATS_COLUMNS_PRE = [
    'rank', 'team', 'age', 'wins', 'losses', 'pythagorean_wins', 'pythagorean_losses', 'margin_of_victory',
    'strength_of_schedule', 'simple_rating_system', 'offensive_rating', 'defensive_rating', 'net_rating',
    'pace', 'free_throw_rate', 'three_point_attempt_rate', 'true_shooting_pct', 
    'offensive_effective_fg_pct', 'offensive_turnover_pct', 'offensive_offensive_rebound_pct', 'offensive_ft_per_fga', 
    'defensive_effective_fg_pct', 'defensive_turnover_pct', 'defensive_defensive_rebound_pct', 'defensive_ft_per_fga', 
    'arena', 'attendance', 'attendance_per_game'
]
ADVANCED_STATS_COLUMNS = [f"advanced_{field}" for field in ADVANCED_STATS_COLUMNS_PRE]

SHOOTING_STATS_COLUMNS_PRE = [
    'rank', 'team', 'games', 'minutes_played', 'field_goal_pct', 'average_shot_distance', 
    'two_point_attempt_pct', 'fga_0_3_pct', 'fga_3_10_pct', 'fga_10_16_pct', 'fga_16_to_3pt_pct', 'three_point_attempt_pct', 
    'two_point_fg_pct', 'fg_0_3_pct', 'fg_3_10_pct', 'fg_10_16_pct', 'fg_16_to_3pt_pct', 'three_point_fg_pct', 
    'two_point_fg_assisted_pct', 'three_point_fg_assisted_pct', 
    'dunks_fga_pct', 'dunks_made', 
    'layups_fga_pct', 'layups_made', 
    'corner_three_pct_fga', 'corner_three_fg_pct', 
    'heaves_attempted', 'heaves_made'
]
SHOOTING_STATS_COLUMNS = [f"shooting_{field}" for field in SHOOTING_STATS_COLUMNS_PRE]

STATS_COLUMNS = TOTALS_STATS_COLUMNS + ADVANCED_STATS_COLUMNS + SHOOTING_STATS_COLUMNS

team_abbreviations = {
    'Oklahoma City Thunder': 'OKC',
    'Miami Heat': 'MIA',
    'Los Angeles Clippers': 'LAC',
    'San Antonio Spurs': 'SAS',
    'Denver Nuggets': 'DEN',
    'New York Knicks': 'NYK',
    'Memphis Grizzlies': 'MEM',
    'Indiana Pacers': 'IND',
    'Houston Rockets': 'HOU',
    'Brooklyn Nets': 'BKN',
    'Los Angeles Lakers': 'LAL',
    'Golden State Warriors': 'GSW',
    'Atlanta Hawks': 'ATL',
    'Chicago Bulls': 'CHI',
    'Utah Jazz': 'UTA',
    'Boston Celtics': 'BOS',
    'Dallas Mavericks': 'DAL',
    'Milwaukee Bucks': 'MIL',
    'Toronto Raptors': 'TOR',
    'Minnesota Timberwolves': 'MIN',
    'Washington Wizards': 'WAS',
    'Portland Trail Blazers': 'POR',
    'Philadelphia 76ers': 'PHI',
    'Detroit Pistons': 'DET',
    'New Orleans Hornets': 'NOH',  # Historical abbreviation before Pelicans
    'Cleveland Cavaliers': 'CLE',
    'Sacramento Kings': 'SAC',
    'Phoenix Suns': 'PHX',
    'Orlando Magic': 'ORL',
    'Charlotte Bobcats': 'CHA',  # Before they became the Hornets again
}
SEASONS = list(range(2013,2024))

# Handle Totals Fields

### Steps:
- Drop blank columns
- Drop 'rank'
- Remove asterisk from 'team' -> append 'made-playoffs'
- Remove 'totals_team' == "League Average"
- Append 'season'
- Append 'team_abbr'
- possibly: normalise

### Final Columns
['totals_team', 'totals_games', 'totals_minutes_played',
       'totals_field_goals', 'totals_field_goals_attempted',
       'totals_field_goal_pct', 'totals_three_point_field_goals',
       'totals_three_point_attempts', 'totals_three_point_pct',
       'totals_two_point_field_goals', 'totals_two_point_attempts',
       'totals_two_point_pct', 'totals_free_throws',
       'totals_free_throw_attempts', 'totals_free_throw_pct',
       'totals_offensive_rebounds', 'totals_defensive_rebounds',
       'totals_total_rebounds', 'totals_assists', 'totals_steals',
       'totals_blocks', 'totals_turnovers', 'totals_personal_fouls',
       'totals_points', 'made_playoffs', 'season', 'team_abbr']

In [2]:
columns = ['totals_team', 'totals_games', 'totals_minutes_played', 'totals_field_goals', 'totals_field_goals_attempted', 'totals_field_goal_pct', 'totals_three_point_field_goals', 'totals_three_point_attempts', 'totals_three_point_pct', 'totals_two_point_field_goals', 'totals_two_point_attempts', 'totals_two_point_pct', 'totals_free_throws', 'totals_free_throw_attempts', 'totals_free_throw_pct', 'totals_offensive_rebounds', 'totals_defensive_rebounds', 'totals_total_rebounds', 'totals_assists', 'totals_steals', 'totals_blocks', 'totals_turnovers', 'totals_personal_fouls', 'totals_points', 'made_playoffs', 'season', 'team_abbr']
final_df_totals = pd.DataFrame(columns=columns)
for season in SEASONS:
    df_totals = pd.read_csv(f'data/TOTALS_STATS/totals_stats_{season}.csv')
    df_totals = df_totals.drop(columns=[col for col in df_totals.columns if (('Unnamed:' in col) or ('totals_rank' in col))])
    df_totals['made_playoffs'] = df_totals['totals_team'].str.contains(r'\*').astype(int)
    df_totals['totals_team'] = df_totals['totals_team'].str.replace(r'\*', '', regex=True)
    df_totals = df_totals[df_totals['totals_team'] != "League Average"]
    df_totals['season'] = season
    df_totals['team_abbr'] = df_totals['totals_team'].map(team_abbreviations)
    final_df_totals = pd.concat([final_df_totals, df_totals], ignore_index=True)

/var/folders/45/fhvy9ynn6s79459s7ys5hmnr0000gp/T/ipykernel_45519/890927339.py:11: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  final_df_totals = pd.concat([final_df_totals, df_totals], ignore_index=True)


In [3]:
final_df_totals = final_df_totals.drop(columns=['totals_team', 'totals_games'])

In [4]:
final_df_totals.to_csv('data/TOTALS_STATS/totals_stats_consolidated.csv', index=False)

In [5]:
final_df_totals

,totals_minutes_played,totals_field_goals,totals_field_goals_attempted,totals_field_goal_pct,totals_three_point_field_goals,totals_three_point_attempts,totals_three_point_pct,totals_two_point_field_goals,totals_two_point_attempts,totals_two_point_pct,...,totals_total_rebounds,totals_assists,totals_steals,totals_blocks,totals_turnovers,totals_personal_fouls,totals_points,made_playoffs,season,team_abbr
0,19905,3339,6983,0.478,521,1518,0.343,2818,5465,0.516,...,3693,2002,762,533,1253,1682,8704,1,2013,DEN
1,19780,3124,6782,0.461,867,2369,0.366,2257,4413,0.511,...,3561,1902,679,359,1348,1662,8688,1,2013,HOU
2,19830,3126,6504,0.481,598,1588,0.377,2528,4916,0.514,...,3579,1753,679,624,1253,1654,8669,1,2013,OKC
3,19880,3210,6675,0.481,663,1764,0.376,2547,4911,0.519,...,3387,2058,695,446,1206,1427,8448,1,2013,SAS
4,19880,3148,6348,0.496,717,1809,0.396,2431,4539,0.536,...,3166,1890,710,441,1143,1533,8436,1,2013,MIA
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
325,19780,3323,7074,0.470,883,2551,0.346,2440,4523,0.539,...,3546,1906,603,382,1236,1652,9136,0,2023,ORL
326,19830,3385,7413,0.457,881,2669,0.330,2504,4744,0.528,...,3652,2062,634,425,1164,1661,9098,0,2023,NaN
327,19755,3329,7287,0.457,856,2619,0.327,2473,4668,0.530,...,3795,1835,600,374,1332,1679,9081,0,2023,HOU
328,19805,3244,7140,0.454,934,2659,0.351,2310,4481,0.516,...,3480,1884,574,308,1237,1813,9045,0,2023,DET


# Handle Advanced Fields

### Steps:
- Drop 'Unnammed' column
- Drop blank columns
- Drop 'rank'
- Remove asterisk from 'team'
- Remove 'totals_team' == "League Average"
- Append 'season'
- Append 'team_abbr'

### Final Columns: 
['advanced_team', 'advanced_age', 'advanced_wins', 'advanced_losses',
       'advanced_pythagorean_wins', 'advanced_pythagorean_losses',
       'advanced_margin_of_victory', 'advanced_strength_of_schedule',
       'advanced_simple_rating_system', 'advanced_offensive_rating',
       'advanced_defensive_rating', 'advanced_net_rating', 'advanced_pace',
       'advanced_free_throw_rate', 'advanced_three_point_attempt_rate',
       'advanced_true_shooting_pct', 'advanced_offensive_effective_fg_pct',
       'advanced_offensive_turnover_pct',
       'advanced_offensive_offensive_rebound_pct',
       'advanced_offensive_ft_per_fga', 'advanced_.1',
       'advanced_defensive_effective_fg_pct',
       'advanced_defensive_turnover_pct',
       'advanced_defensive_defensive_rebound_pct',
       'advanced_defensive_ft_per_fga', 'advanced_.2', 'advanced_arena',
       'advanced_attendance', 'advanced_attendance_per_game', 'season',
       'team_abbr']

In [6]:
columns = ['advanced_team', 'advanced_age', 'advanced_wins', 'advanced_losses', 'advanced_pythagorean_wins', 'advanced_pythagorean_losses', 'advanced_margin_of_victory', 'advanced_strength_of_schedule', 'advanced_simple_rating_system', 'advanced_offensive_rating', 'advanced_defensive_rating', 'advanced_net_rating', 'advanced_pace', 'advanced_free_throw_rate', 'advanced_three_point_attempt_rate', 'advanced_true_shooting_pct', 'advanced_offensive_effective_fg_pct', 'advanced_offensive_turnover_pct', 'advanced_offensive_offensive_rebound_pct', 'advanced_offensive_ft_per_fga', 'advanced_.1', 'advanced_defensive_effective_fg_pct', 'advanced_defensive_turnover_pct', 'advanced_defensive_defensive_rebound_pct', 'advanced_defensive_ft_per_fga', 'advanced_.2', 'advanced_arena', 'advanced_attendance', 'advanced_attendance_per_game', 'season', 'team_abbr']
df_advanced_final = pd.DataFrame(columns=columns)
for season in SEASONS:
    df_advanced = pd.read_csv(f'data/ADVANCED_STATS/advanced_stats_{season}.csv')
    df_advanced = df_advanced.drop(columns=[col for col in df_advanced.columns if (('Unnamed:' in col) or ('advanced_rank' in col))])
    df_advanced = df_advanced.loc[:, ~df_advanced.columns.str.endswith('advanced_')]
    df_advanced['advanced_team'] = df_advanced['advanced_team'].str.replace(r'\*', '', regex=True)
    df_advanced = df_advanced[df_advanced['advanced_team'] != "League Average"]
    df_advanced = df_advanced[df_advanced['advanced_team'] != "Team"]
    df_advanced['season'] = season
    df_advanced['team_abbr'] = df_advanced['advanced_team'].map(team_abbreviations)
    df_advanced.sort_values(by='advanced_team')
    
    df_advanced_final = pd.concat([df_advanced_final, df_advanced], ignore_index=True)

In [7]:
df_advanced_final = df_advanced_final.drop(columns=['advanced_.1', 'advanced_.2', 'advanced_team'])

In [8]:
df_advanced_final.to_csv('data/ADVANCED_STATS/advanced_stats_consolidated.csv', index=False)

# Handle Shooting Fields

### Steps:
- Drop Unnamed column
- Drop blank columns
- Drop 'rank'
- Remove first row
- Remove asterisk from 'team'
- Append 'season'
- Append 'team_abbr'

### Final Columns:
['shooting_team', 'shooting_games', 'shooting_minutes_played',
       'shooting_field_goal_pct', 'shooting_average_shot_distance',
       'shooting_two_point_attempt_pct', 'shooting_fga_0_3_pct',
       'shooting_fga_3_10_pct', 'shooting_fga_10_16_pct',
       'shooting_fga_16_to_3pt_pct', 'shooting_three_point_attempt_pct',
       'shooting_.1', 'shooting_two_point_fg_pct', 'shooting_fg_0_3_pct',
       'shooting_fg_3_10_pct', 'shooting_fg_10_16_pct',
       'shooting_fg_16_to_3pt_pct', 'shooting_three_point_fg_pct',
       'shooting_.2', 'shooting_two_point_fg_assisted_pct',
       'shooting_three_point_fg_assisted_pct', 'shooting_.3',
       'shooting_dunks_fga_pct', 'shooting_dunks_made', 'shooting_.4',
       'shooting_layups_fga_pct', 'shooting_layups_made', 'shooting_.5',
       'shooting_corner_three_pct_fga', 'shooting_corner_three_fg_pct',
       'shooting_.6', 'shooting_heaves_attempted', 'shooting_heaves_made',
       'season', 'team_abbr']

In [9]:
columns = ['shooting_team', 'shooting_games', 'shooting_minutes_played', 'shooting_field_goal_pct', 'shooting_average_shot_distance', 'shooting_two_point_attempt_pct', 'shooting_fga_0_3_pct', 'shooting_fga_3_10_pct', 'shooting_fga_10_16_pct', 'shooting_fga_16_to_3pt_pct', 'shooting_three_point_attempt_pct', 'shooting_.1', 'shooting_two_point_fg_pct', 'shooting_fg_0_3_pct', 'shooting_fg_3_10_pct', 'shooting_fg_10_16_pct', 'shooting_fg_16_to_3pt_pct', 'shooting_three_point_fg_pct', 'shooting_.2', 'shooting_two_point_fg_assisted_pct', 'shooting_three_point_fg_assisted_pct', 'shooting_.3', 'shooting_dunks_fga_pct', 'shooting_dunks_made', 'shooting_.4', 'shooting_layups_fga_pct', 'shooting_layups_made', 'shooting_.5', 'shooting_corner_three_pct_fga', 'shooting_corner_three_fg_pct', 'shooting_.6', 'shooting_heaves_attempted', 'shooting_heaves_made', 'season', 'team_abbr']
df_shooting_final = pd.DataFrame(columns=columns)
for season in SEASONS:
    df_shooting = pd.read_csv(f'data/SHOOTING_STATS/shooting_stats_{season}.csv')
    df_shooting = df_shooting.drop(columns=[col for col in df_shooting.columns if (('Unnamed:' in col) or ('shooting_rank' in col))])
    df_shooting = df_shooting.loc[:, ~df_shooting.columns.str.endswith('shooting_')]
    df_shooting['shooting_team'] = df_shooting['shooting_team'].str.replace(r'\*', '', regex=True)
    df_shooting = df_shooting[df_shooting['shooting_team'] != "League Average"]
    df_shooting = df_shooting[df_shooting['shooting_team'] != "Team"]
    df_shooting['season'] = season
    df_shooting['team_abbr'] = df_shooting['shooting_team'].map(team_abbreviations)
    df_shooting_final = pd.concat([df_shooting_final, df_shooting], ignore_index=True)

In [10]:
df_shooting_final = df_shooting_final.drop(columns=['shooting_.1', 'shooting_.2', 'shooting_.3', 'shooting_.4', 'shooting_.5', 'shooting_.6', 'shooting_team', 'shooting_games'])

In [11]:
df_shooting_final.to_csv('data/SHOOTING_STATS/shooting_stats_consolidated.csv', index=False)

# Putting the 3 Tables Together

In [12]:
merged_totals_advanced_df = pd.merge(final_df_totals, df_advanced_final, on=['team_abbr', 'season'], how='outer')
all_stats_final_df = pd.merge(merged_totals_advanced_df, df_shooting_final, on=['team_abbr', 'season'], how='outer')

In [13]:
all_stats_final_df.to_csv('data/all_stats.csv', index=False)

## Append 'home_' and 'away_' to all fields from which the home_df will merge

In [14]:
home_stats_df = all_stats_final_df
home_stats_df = home_stats_df.rename(columns=lambda x: f'home_{x}' if x not in ['season'] else x)
away_stats_df = all_stats_final_df
away_stats_df = away_stats_df.rename(columns=lambda x: f'away_{x}' if x not in ['season'] else x)

# Putting Match data with Stats

In [15]:
from helper import run_sql

In [36]:
SQL_query = """
            SELECT g.team_name_home, g.team_abbreviation_home AS home_team_abbr, g.team_name_away, g.team_abbreviation_away AS away_team_abbr, DATE( g.game_date ) as game_date, cast(strftime('%Y', g.game_date) AS int) AS season, g.season_type, cast(g.plus_minus_home AS int) AS plus_minus_home, g.game_id
            FROM game g 
            WHERE DATE( g.game_date ) > DATE('2014-10-22')
            AND g.team_abbreviation_home IN ("OKC", "MIA", "LAC", "SAS", "DEN", "NYK", "MEM", "IND", "HOU", "BKN", "LAL", "GSW", "ATL", "CHI", "UTA", "BOS", "DAL", "MIL", "TOR", "MIN", "WAS", "POR", "PHI", "DET", "NOP", "CLE", "SAC", "PHX", "ORL", "CHA")
            AND g.team_abbreviation_away IN ("OKC", "MIA", "LAC", "SAS", "DEN", "NYK", "MEM", "IND", "HOU", "BKN", "LAL", "GSW", "ATL", "CHI", "UTA", "BOS", "DAL", "MIL", "TOR", "MIN", "WAS", "POR", "PHI", "DET", "NOP", "CLE", "SAC", "PHX", "ORL", "CHA")
            """

db_path = 'nba.sqlite'

matches_df = run_sql(SQL_query, db_path, verbose=False)
matches_df.to_csv('data/matches_data.csv', index=False)

## Create Final training data
NOTE: remember to map current pt_diff results against the relevant team's <b>t-1</b> stats

In [113]:
i = 1
training_df = pd.DataFrame()
for index, row in matches_df.iterrows():
    print(f"Concat where home: {row.home_team_abbr} | away: {row.away_team_abbr} | season: {row.season} | entry no: {i}")
    i+=1

    row_from_matches_df = matches_df[(matches_df['game_id'] == row.game_id)]

    row_from_home_stats = home_stats_df[
        (home_stats_df['home_team_abbr'] == row.home_team_abbr) & 
        (home_stats_df['season'] == int(row.season) - 1)
    ]
    # row_from_home_stats['season'] = pd.to_numeric(row_from_home_stats['season'], errors='coerce').astype(int)
    
    row_from_away_stats = away_stats_df[
        (away_stats_df['away_team_abbr'] == row.away_team_abbr) & 
        (away_stats_df['season'] == int(row.season) - 1)
    ]
    # row_from_away_stats['season'] = pd.to_numeric(row_from_away_stats['season'], errors='coerce').astype(int)

    
    _combined_pre = pd.merge(row_from_home_stats, row_from_matches_df, on=['home_team_abbr'], how='inner')
    combined_row = pd.merge(_combined_pre, row_from_away_stats, on=['away_team_abbr'], how='inner')

    training_df = pd.concat([training_df, combined_row])
    

Concat where home: SAS | away: DAL | season: 2014 | entry no: 1
Concat where home: LAL | away: HOU | season: 2014 | entry no: 2
Concat where home: NOP | away: ORL | season: 2014 | entry no: 3
Concat where home: TOR | away: ATL | season: 2014 | entry no: 4
Concat where home: MIA | away: WAS | season: 2014 | entry no: 5
Concat where home: POR | away: OKC | season: 2014 | entry no: 6
Concat where home: BOS | away: BKN | season: 2014 | entry no: 7
Concat where home: UTA | away: HOU | season: 2014 | entry no: 8
Concat where home: CHA | away: MIL | season: 2014 | entry no: 9
Concat where home: PHX | away: LAL | season: 2014 | entry no: 10
Concat where home: SAC | away: GSW | season: 2014 | entry no: 11
Concat where home: NYK | away: CHI | season: 2014 | entry no: 12
Concat where home: IND | away: PHI | season: 2014 | entry no: 13
Concat where home: DEN | away: DET | season: 2014 | entry no: 14
Concat where home: LAC | away: OKC | season: 2014 | entry no: 15
Concat where home: ORL | away: WAS

In [114]:
training_df

,home_totals_minutes_played,home_totals_field_goals,home_totals_field_goals_attempted,home_totals_field_goal_pct,home_totals_three_point_field_goals,home_totals_three_point_attempts,home_totals_three_point_pct,home_totals_two_point_field_goals,home_totals_two_point_attempts,home_totals_two_point_pct,...,away_shooting_two_point_fg_assisted_pct,away_shooting_three_point_fg_assisted_pct,away_shooting_dunks_fga_pct,away_shooting_dunks_made,away_shooting_layups_fga_pct,away_shooting_layups_made,away_shooting_corner_three_pct_fga,away_shooting_corner_three_fg_pct,away_shooting_heaves_attempted,away_shooting_heaves_made
0,19880,3210,6675,0.481,663,1764,0.376,2547,4911,0.519,...,.538,.860,.031,196,.198,813,.232,.363,18,0
0,19755,3041,6640,0.458,715,2015,0.355,2326,4625,0.503,...,.520,.841,.066,415,.280,1091,.290,.392,11,2
0,19980,2979,6685,0.446,571,1665,0.343,2408,5020,0.480,...,.585,.874,.052,323,.222,924,.264,.426,12,0
0,19880,3148,6348,0.496,717,1809,0.396,2431,4539,0.536,...,.543,.901,.047,296,.188,675,.330,.457,8,0
0,19855,3009,6715,0.448,673,1904,0.353,2336,4811,0.486,...,.492,.851,.071,418,.207,812,.252,.440,13,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
0,19805,3416,7079,0.483,1039,2944,0.353,2377,4135,0.575,...,.537,.849,.049,304,.236,941,.296,.418,23,0
0,19805,3416,7079,0.483,1039,2944,0.353,2377,4135,0.575,...,.537,.849,.049,304,.236,941,.296,.418,23,0
0,19855,3246,6954,0.467,1114,2936,0.379,2132,4018,0.531,...,.584,.857,.059,373,.266,1128,.265,.384,24,0
0,19855,3246,6954,0.467,1114,2936,0.379,2132,4018,0.531,...,.584,.857,.059,373,.266,1128,.265,.384,24,0


In [112]:
training_df.to_csv('training_data.csv', index=False)